In [1]:
import cv2

In [7]:
# load video path: sample_videos\Autonomous driving lane detection sample video 1.mp4

video_path = "sample_videos\Autonomous driving lane detection sample video 1.mp4"
cap = cv2.VideoCapture(video_path)


# change image frame to gray scale
def gray_scale(frame):
    return cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# apply gaussian blur to the image
def gaussian_blur(frame):
    return cv2.GaussianBlur(frame, (5, 5), 0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    # convert to gray scale
    gray = gray_scale(frame)

    # apply gaussian blur
    blur = gaussian_blur(gray)

    # show the video
    cv2.imshow('frame', blur)

    # video 60 fps, wait according to fps
    cv2.waitKey(int(1000/60))

    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break



